In [1]:
from pyscf import gto, scf, cc, lib
import numpy as np
from functools import partial
import os

np.einsum = partial(np.einsum, optimize=True)
np.set_printoptions(6, suppress=True, linewidth=150)

## System Definition and Reference CC

In [2]:
mol = gto.Mole(atom="O; H 1 0.94; H 1 0.94 2 104.5", basis="cc-pVDZ").build()

In [3]:
# this is not the correct way to perform RI-HF with RI-CCSD, these two methods should use different auxbasis
# however, energy difference is actually small
# and we only want to demonstrate correctness and efficiency of program, so which auxbasis is not really matters
mf = scf.RHF(mol).density_fit(auxbasis="cc-pVDZ-ri").run()

converged SCF energy = -76.0280410408039


In [4]:
mol.verbose = 4

In [5]:
mf_cc = cc.CCSD(mf).run()


******** <class 'pyscf.cc.dfccsd.RCCSD'> ********
CC2 = 0
CCSD nocc = 5, nmo = 24
max_cycle = 50
direct = 0
conv_tol = 1e-07
conv_tol_normt = 1e-05
diis_space = 6
diis_start_cycle = 0
diis_start_energy_diff = 1e+09
max_memory 4000 MB (current use 158 MB)
Init t2, MP2 energy = -76.2307949519314  E_corr(MP2) -0.20275391112758
Init E_corr(RCCSD) = -0.20275391112864
cycle = 1  E_corr(RCCSD) = -0.207993731903545  dE = -0.00523982077  norm(t1,t2) = 0.0209855
cycle = 2  E_corr(RCCSD) = -0.211098740172299  dE = -0.00310500827  norm(t1,t2) = 0.00713562
cycle = 3  E_corr(RCCSD) = -0.212120511921666  dE = -0.00102177175  norm(t1,t2) = 0.00266794
cycle = 4  E_corr(RCCSD) = -0.21223592969047  dE = -0.000115417769  norm(t1,t2) = 0.000469925
cycle = 5  E_corr(RCCSD) = -0.212219794258673  dE = 1.61354318e-05  norm(t1,t2) = 0.000154335
cycle = 6  E_corr(RCCSD) = -0.21221718589628  dE = 2.60836239e-06  norm(t1,t2) = 4.34129e-05
cycle = 7  E_corr(RCCSD) = -0.212218507545593  dE = -1.32164931e-06  norm(t

<class 'pyscf.cc.dfccsd.RCCSD'> does not have attributes  converged


cycle = 9  E_corr(RCCSD) = -0.212217982473886  dE = 9.58360419e-09  norm(t1,t2) = 6.94985e-07
RCCSD converged
E(RCCSD) = -76.24025902327774  E_corr = -0.2122179824738857


## Global Variables

In [6]:
cderi_ao = mf.with_df._cderi
mo_occ = mf.mo_occ
mo_energy = mf.mo_energy
mo_coeff = mf.mo_coeff

nmo = len(mo_occ)
nocc = (mo_occ > 0).sum()
nvir = nmo - nocc
nao = mol.nao
naux = cderi_ao.shape[0]

so = slice(0, nocc)
sv = slice(nocc, nmo)

In [7]:
B = np.einsum("Puv, up, vq -> pqP", lib.unpack_tril(cderi_ao), mo_coeff, mo_coeff)
D_ov = mo_energy[so, None] - mo_energy[None, sv]

## Functions

In [8]:
def get_riccsd_intermediates_1(intermediates, B, t1, t2):
    # M1j = np.einsum("jbP, jb -> P", B[so, sv], t1)
    M1j = t1.reshape(-1) @ B[so, sv].reshape([-1, naux])
    
    # M1oo = np.einsum("jaP, ia -> ijP", B[so, sv], t1)
    M1oo = np.empty([nocc, nocc, naux])
    for j in range(nocc):  # par
        M1oo[:, j] = t1 @ B[j, sv]
    
    # M2a = np.einsum("jbP, ijab -> iaP", B[so, sv], (2 * t2 - t2.swapaxes(-1, -2)))
    M2a = np.empty([nocc, nvir, naux])
    for i in range(nocc):  # par
        scr_jba = - t2[i] + 2 * t2.swapaxes(-1, -2)[i]
        M2a[i] = scr_jba.reshape([-1, nvir]).T @ B[so, sv].reshape([-1, naux])

    intermediates.update({
        "M1j": M1j,
        "M1oo": M1oo,
        "M2a": M2a,
    })

In [9]:
def get_riccsd_intermediates_2(intermediates, B, t1, t2):
    M1oo = intermediates["M1oo"]
    
    # M1aov = np.einsum("ijP, ja -> iaP", B[so, so], t1)
    M1aov = np.empty([nocc, nvir, naux])
    for i in range(nocc):  # par
        M1aov[i] = t1.T @ B[i, so]
    
    # M1bov  = np.einsum("abP, ib -> iaP", B[sv, sv], t1)
    M1bov = (t1 @ B[sv, sv].reshape([nvir, -1])).reshape([nocc, nvir, naux])
    
    # M1vv = np.einsum("ibP, ia -> abP", B[so, sv], t1)
    M1vv = (t1.T @ B[so, sv].reshape([nocc, -1])).reshape([nvir, nvir, naux])
    
    # M2b = np.einsum("ikP, ka -> iaP", M1oo, t1)
    M2b = np.empty([nocc, nvir, naux])
    for i in range(nocc):
        M2b[i] = t1.T @ M1oo[i]

    intermediates.update({
        "M1aov": M1aov,
        "M1bov": M1bov,
        "M1vv": M1vv,
        "M2b": M2b,
    })

In [10]:
def get_riccsd_residue_1(RHS1, intermediates, B, t1, t2):

    M1j = intermediates["M1j"]
    M1aov = intermediates["M1aov"]
    M1bov = intermediates["M1bov"]
    M1oo = intermediates["M1oo"]
    M1vv = intermediates["M1vv"]
    M2a = intermediates["M2a"]
    M2b = intermediates["M2b"]
    
    # === TERM1 === #
    # RHS1 += - 1 * np.einsum("lcP, lkP, ikac -> ia", B[so, sv], M1oo, (2 * t2 - t2.swapaxes(-1, -2)))
    
    # "lcP, lkP -> kc", B[so, sv], M1oo
    scr_kc = np.zeros([nocc, nvir])
    for l in range(nocc):
        scr_kc += M1oo[l] @ B[l, sv].T
    
    # "kc, ikac -> ia", scr_kc, (2 * t2 - t2.swapaxes(-1, -2)))
    for i in range(nocc):
        t2_i = 2 * t2[i] - t2[i].swapaxes(-1, -2)
        t2_i *= scr_kc[:, None, :]
        RHS1[i] -= t2_i.sum(axis=(0, 2))
    
    # === TERM2 === #
    # RHS1 += - 1 * np.einsum("kcP, icP, ka -> ia", B[so, sv], (M2a - M1aov), t1)
    
    RHS1 -= (M2a - M1aov).reshape([nocc, -1]) @ B[so, sv].reshape([nocc, -1]).T @ t1
    
    # === TERM3 === #
    # RHS1 +=   1 * np.einsum("icP, acP -> ia", M2a, B[sv, sv])
    
    RHS1 += M2a.reshape([nocc, -1]) @ B[sv, sv].reshape([nvir, -1]).T
    
    # === TERM4 === #
    # RHS1 += - 1 * np.einsum("ikP, kaP -> ia", (B[so, so] + M1oo), (M2a + M1bov))
    
    for k in range(nocc):
        RHS1 -= (B[so, k] + M1oo[:, k]) @ (M2a[k] + M1bov[k]).T
    
    # === TERM5 === #
    # RHS1 +=   2 * np.einsum("iaP, P -> ia", (B[so, sv] + M1bov - M1aov + M2a - 0.5 * M2b), M1j)
    
    scr_iaP = B[so, sv] + M1bov - M1aov + M2a - 0.5 * M2b
    RHS1 += 2 * (scr_iaP.reshape([-1, naux]) @ M1j).reshape([nocc, nvir])

In [11]:
def get_riccsd_residue_2_lt2_contract(RHS2, intermediates, B, t1, t2):

    M1j = intermediates["M1j"]
    M1aov = intermediates["M1aov"]
    M1bov = intermediates["M1bov"]
    M1oo = intermediates["M1oo"]
    M1vv = intermediates["M1vv"]
    M2a = intermediates["M2a"]
    M2b = intermediates["M2b"]
    
    # ====== Loo, Lvv ====== #
    
    Loo = M2a.reshape([nocc, -1]) @ B[so, sv].reshape([nocc, -1]).T
    Loo += ((2 * B[so, so] + M1oo).reshape([-1, naux]) @ M1j).reshape([nocc, nocc])
    for l in range(nocc):  # non-par
        Loo -= B[l, so] @ M1oo[l].T
    
    # RHS2 -= np.einsum("ik, kjab -> ijab", Loo, t2)
    RHS2 -= (Loo @ t2.reshape([nocc, -1])).reshape([nocc, nocc, nvir, nvir])
    
    Lvv = ((2 * B[sv, sv] - M1vv).reshape([-1, naux]) @ M1j).reshape([nvir, nvir])
    for k in range(nocc):  # non-par
        Lvv -= (M2a + M1bov)[k] @ B[k, sv].T
    
    # RHS2 += np.einsum("bc, ijac -> ijab", Lvv, t2)
    RHS2 += (t2.reshape([-1, nvir]) @ Lvv.T).reshape([nocc, nocc, nvir, nvir])

In [12]:
def get_riccsd_residue_2_direct_dot(RHS2, intermediates, B, t1, t2):

    M1j = intermediates["M1j"]
    M1aov = intermediates["M1aov"]
    M1bov = intermediates["M1bov"]
    M1oo = intermediates["M1oo"]
    M1vv = intermediates["M1vv"]
    M2a = intermediates["M2a"]
    M2b = intermediates["M2b"]
    
    # ====== Direct Dot ====== #
    
    # RHS2 += np.einsum("iaP, jbP -> ijab", B[so, sv] + M2a, 0.5 * B[so, sv] + 0.5 * M2a - M1aov + M1bov - M2b)
    # RHS2 -= np.einsum("iaP, jbP -> ijab", M1aov, M1bov)
    
    scr_iaP = B[so, sv] + M2a
    scr_jbP = 0.5 * B[so, sv] + 0.5 * M2a - M1aov + M1bov - M2b
    
    for i in range(nocc):  # par
        for j in range(nocc):  # par
            RHS2[i, j] += scr_iaP[i] @ scr_jbP[j].T
            RHS2[i, j] -= M1aov[i] @ M1bov[j].T

In [13]:
def get_riccsd_residue_2_o3v3(RHS2, intermediates, B, t1, t2):

    M1j = intermediates["M1j"]
    M1aov = intermediates["M1aov"]
    M1bov = intermediates["M1bov"]
    M1oo = intermediates["M1oo"]
    M1vv = intermediates["M1vv"]
    M2a = intermediates["M2a"]
    M2b = intermediates["M2b"]
    
    # ====== O3V3 ====== #
    
    # t2t = np.ascontiguousarray(t2.swapaxes(-1, -2))
    # scr1 = np.einsum("kdP, lcP -> kldc", B[so, sv], B[so, sv])
    # scr2 = np.einsum("kldc, ilda -> ikca", scr1, t2)
    # scr3 = np.einsum("kldc, ilda -> ikca", scr1, t2t)
    # scr4 = np.einsum("ikP, acP -> ikca", (B[so, so] + M1oo), (B[sv, sv] - M1vv))
    # RHS2 += np.einsum("ikca, jkcb -> ijab", - scr4 + scr2 - scr3, t2t)
    # RHS2 += np.einsum("ikcb, jkca -> ijab", - scr4 + 0.5 * scr2, t2)
    
    scr_ikP = (B[so, so] + M1oo)
    scr_acP = (B[sv, sv] - M1vv)
    t2t = np.ascontiguousarray(t2.swapaxes(-1, -2))
    
    scr1 = np.zeros([nocc, nocc, nvir, nvir])
    for k in range(nocc):
        for l in range(k + 1):
            scr = B[k, sv] @ B[l, sv].T
            scr1[k, l] = scr
            if k != l:
                scr1[l, k] = scr.T
    
    for i in range(nocc):
        scr2 = np.zeros([nocc, nvir, nvir])
        scr3 = np.zeros([nocc, nvir, nvir])
        for k in range(nocc):
            scr2[k] = scr1[k].reshape([-1, nvir]).T @ t2[i].reshape([-1, nvir])
            scr3[k] = scr1[k].reshape([-1, nvir]).T @ t2t[i].reshape([-1, nvir])
        scr4 = np.zeros([nocc, nvir, nvir])
        for c in range(nvir):
            scr4[:, c] = scr_ikP[i] @ scr_acP[:, c].T
        scr5 = - scr3 - scr4 + scr2
        for j in range(nocc):  # par
            RHS2[i, j] += scr5.reshape([-1, nvir]).T @ t2t[j].reshape([-1, nvir])
        scr5 = - scr4 + 0.5 * scr2
        for j in range(nocc):  # par
            RHS2[i, j] += t2[j].reshape([-1, nvir]).T @ scr5.reshape([-1, nvir])

In [14]:
def get_riccsd_residue_2_o4v2(RHS2, intermediates, B, t1, t2):

    M1j = intermediates["M1j"]
    M1aov = intermediates["M1aov"]
    M1bov = intermediates["M1bov"]
    M1oo = intermediates["M1oo"]
    M1vv = intermediates["M1vv"]
    M2a = intermediates["M2a"]
    M2b = intermediates["M2b"]
    
    # ====== O4V2 (HHL) ====== #
    
    # Woooo = 0
    # Woooo += np.einsum("ikP, jlP         -> ijkl", B[so, so] + M1oo, B[so, so] + M1oo)
    # Woooo += np.einsum("kcP, ldP, ijcd   -> ijkl", B[so, sv], B[so, sv], t2)
    # RHS2 += 0.5 * np.einsum("ijkl, klab   -> ijab", Woooo, t2,   )
    # RHS2 += 0.5 * np.einsum("ijkl, ka, lb -> ijab", Woooo, t1, t1)
    
    scr_BM1oo = B[so, so] + M1oo
    
    scr_ijkl = np.zeros([nocc, nocc, nocc, nocc])
    for i in range(nocc):
        for j in range(i + 1):
            tmp_ab = scr_BM1oo[i] @ scr_BM1oo[j].T
            scr_ijkl[i, j] = tmp_ab
            scr_ijkl[j, i] = tmp_ab.T
    scr_klcd = np.zeros([nocc, nocc, nvir, nvir])
    for k in range(nocc):
        for l in range(k + 1):
            tmp_cd = B[k, sv] @ B[l, sv].T
            scr_klcd[k, l] = tmp_cd
            scr_klcd[l, k] = tmp_cd.T
    scr_ijkl += (t2.reshape([nocc * nocc, -1]) @ scr_klcd.reshape([nocc * nocc, -1]).T).reshape([nocc, nocc, nocc, nocc])
    
    tau2 = t2 + t1[:, None, :, None] * t1[None, :, None, :]
    RHS2 += 0.5 * (scr_ijkl.reshape([nocc * nocc, -1]) @ tau2.reshape([-1, nvir * nvir])).reshape(t2.shape)

In [15]:
def get_riccsd_residue_2_o2v4(RHS2, intermediates, B, t1, t2):

    M1j = intermediates["M1j"]
    M1aov = intermediates["M1aov"]
    M1bov = intermediates["M1bov"]
    M1oo = intermediates["M1oo"]
    M1vv = intermediates["M1vv"]
    M2a = intermediates["M2a"]
    M2b = intermediates["M2b"]
    
    # ====== O2V4 (PPL) ====== #
    
    # Wvvvv = 0
    # Wvvvv += np.einsum("acP, bdP -> abcd", B[sv, sv] - M1vv, B[sv, sv] - M1vv)
    # Wvvvv -= np.einsum("acP, bdP -> abcd", M1vv, M1vv)
    # RHS2 += 0.5 * np.einsum("abcd, ijcd   -> ijab", Wvvvv, t2,   )
    # RHS2 += 0.5 * np.einsum("abcd, ic, jd -> ijab", Wvvvv, t1, t1)
    
    # scr_acP = (B[sv, sv] - M1vv)
    
    # for a in range(nvir):
    #     for b in range(a + 1):
    #         tmp_cd = scr_acP[a] @ scr_acP[b].T
    #         tmp_cd -= M1vv[a] @ M1vv[b].T
    #         tmp_ij = 0.5 * tau2.reshape([nocc, nocc, nvir * nvir]) @ tmp_cd.reshape(-1)
    #         RHS2[:, :, a, b] += tmp_ij
    #         if a != b:
    #             RHS2[:, :, b, a] += tmp_ij.T
    
    RHS_PPL = np.zeros([nocc, nocc, nvir, nvir])
    
    scr_acP = (B[sv, sv] - M1vv)
    tau2 = t2 + t1[:, None, :, None] * t1[None, :, None, :]
    
    batch_a = 8
    batch_b = 32
    for a_start in range(0, nvir, batch_a):  # par
        a_end = min(a_start + batch_a, nvir)
        nbatch_a = a_end - a_start
        sa = slice(a_start, a_end)
        for b_start in range(0, a_end, batch_b):  # par
            b_end = min(b_start + batch_b, a_end)
            nbatch_b = b_end - b_start
            sb = slice(b_start, b_end)
    
            scr_abcd = np.zeros([nbatch_a, nbatch_b, nvir, nvir])
            for a in range(nbatch_a):  # par
                for b in range(nbatch_b):  # par
                    scr_abcd[a, b] += scr_acP[a + a_start] @ scr_acP[b + b_start].T
                    scr_abcd[a, b] -= M1vv[a + a_start] @ M1vv[b + b_start].T
            scr_ijab = 0.5 * tau2.reshape([nocc * nocc, nvir * nvir]) @ scr_abcd.reshape([nbatch_a * nbatch_b, nvir * nvir]).T
            scr_ijab = scr_ijab.reshape([nocc, nocc, nbatch_a, nbatch_b])
            RHS_PPL[:, :, sa, sb] = scr_ijab
    for i in range(nocc):  # par
        for j in range(nocc):  # par
            for a in range(nvir):
                for b in range(a):
                    RHS_PPL[i, j, b, a] = RHS_PPL[j, i, a, b]
    
    RHS2 += RHS_PPL

In [16]:
def get_riccsd_energy(intermediates, B, t1, t2):
    M1j = intermediates["M1j"]
    M1oo = intermediates["M1oo"]
    M2a = intermediates["M2a"]

    e_t1 = 2 * M1j.reshape(-1) @ M1j.reshape(-1)
    e_t1 -= M1oo.reshape(-1) @ M1oo.swapaxes(0, 1).reshape(-1)
    e_t2 = B[so, sv].reshape(-1) @ M2a.reshape(-1)
    return e_t1, e_t2

In [17]:
def get_riccsd_amplitude(intermediates, B, mo_energy, e_corr, t1, t2):
    get_riccsd_intermediates_2(intermediates, B, mf_cc.t1, mf_cc.t2)

    RHS1 = np.zeros([nocc, nvir])
    get_riccsd_residue_1(RHS1, intermediates, B, mf_cc.t1, mf_cc.t2)

    RHS2 = np.zeros([nocc, nocc, nvir, nvir])
    get_riccsd_residue_2_lt2_contract(RHS2, intermediates, B, mf_cc.t1, mf_cc.t2)
    get_riccsd_residue_2_direct_dot(RHS2, intermediates, B, mf_cc.t1, mf_cc.t2)
    get_riccsd_residue_2_o3v3(RHS2, intermediates, B, mf_cc.t1, mf_cc.t2)
    get_riccsd_residue_2_o2v4(RHS2, intermediates, B, mf_cc.t1, mf_cc.t2)
    get_riccsd_residue_2_o4v2(RHS2, intermediates, B, mf_cc.t1, mf_cc.t2)
    # RHS2 = RHS2 + RHS2.transpose((1, 0, 3, 2))

    # update t1
    D_ov = mo_energy[so, None] - mo_energy[None, sv]
    RHS1 /= D_ov
    t1_new = RHS1

    # update t2
    for i in range(nocc):  # par
        for j in range(i + 1):  # par
            t2_ab = RHS2[i, j] + RHS2[j, i].T
            d2_ab = (D_ov[i][:, None] + D_ov[j][None, :])
            t2_ab /= d2_ab
            RHS2[i, j] = t2_ab
            if i != j:
                RHS2[j, i] = t2_ab.T
    t2_new = RHS2

    delta_t1 = np.abs(t1_new - t1).sum()
    delta_t2 = np.abs(t2_new - t2).sum()

    get_riccsd_intermediates_1(intermediates, B, t1_new, t2_new)
    e1_new, e2_new = get_riccsd_energy(intermediates, B, t1_new, t2_new)
    e_corr_new = e1_new + e2_new

    delta_e_corr = np.abs(e_corr_new - e_corr)

    print(f"delta_e_corr {delta_e_corr}")
    print(f"delta_t1     {delta_t1}")
    print(f"delta_t2     {delta_t2}")
    print(f"e_corr       {e_corr_new}")
    
    return (e_corr_new, t1_new, t2_new), (delta_e_corr, delta_t1, delta_t2)

In [18]:
def get_riccsd_amplitude(intermediates, B, mo_energy, e_corr, t1, t2):
    get_riccsd_intermediates_2(intermediates, B, t1, t2)

    RHS1 = np.zeros([nocc, nvir])
    get_riccsd_residue_1(RHS1, intermediates, B, t1, t2)

    RHS2 = np.zeros([nocc, nocc, nvir, nvir])
    get_riccsd_residue_2_lt2_contract(RHS2, intermediates, B, t1, t2)
    get_riccsd_residue_2_direct_dot(RHS2, intermediates, B, t1, t2)
    get_riccsd_residue_2_o3v3(RHS2, intermediates, B, t1, t2)
    get_riccsd_residue_2_o2v4(RHS2, intermediates, B, t1, t2)
    get_riccsd_residue_2_o4v2(RHS2, intermediates, B, t1, t2)
    # RHS2 = RHS2 + RHS2.transpose((1, 0, 3, 2))

    # update t1
    D_ov = mo_energy[so, None] - mo_energy[None, sv]
    RHS1 /= D_ov
    t1_new = RHS1

    # update t2
    for i in range(nocc):  # par
        for j in range(i + 1):  # par
            t2_ab = RHS2[i, j] + RHS2[j, i].T
            d2_ab = (D_ov[i][:, None] + D_ov[j][None, :])
            t2_ab /= d2_ab
            RHS2[i, j] = t2_ab
            if i != j:
                RHS2[j, i] = t2_ab.T
    t2_new = RHS2

    delta_t1 = np.abs(t1_new - t1).sum()
    delta_t2 = np.abs(t2_new - t2).sum()

    get_riccsd_intermediates_1(intermediates, B, t1_new, t2_new)
    e1_new, e2_new = get_riccsd_energy(intermediates, B, t1_new, t2_new)
    e_corr_new = e1_new + e2_new

    delta_e_corr = np.abs(e_corr_new - e_corr)

    print(f"delta_e_corr {delta_e_corr}")
    print(f"delta_t1     {delta_t1}")
    print(f"delta_t2     {delta_t2}")
    print(f"e_corr       {e_corr_new}")
    
    return (e_corr_new, t1_new, t2_new), (delta_e_corr, delta_t1, delta_t2)

## Actual CCSD Iterations

In [19]:
intermediates = {
    "M1j": None,
    "M1oo": None,
    "M1aov": None,
    "M1bov": None,
    "M1vv": None,
    "M2a": None,
    "M2b": None,
}

get_riccsd_intermediates_1(intermediates, B, mf_cc.t1, mf_cc.t2)
get_riccsd_amplitude(intermediates, B, mo_energy, 0, mf_cc.t1, mf_cc.t2);

delta_e_corr 0.21221786522503241
delta_t1     1.5344182097907008e-06
delta_t2     8.552378380389298e-06
e_corr       -0.21221786522503241


In [20]:
t1 = np.zeros([nocc, nvir])
t2 = np.zeros([nocc, nocc, nvir, nvir])
e_corr = 0
for i in range(nocc):
    for j in range(i + 1):
        d2_ab = (D_ov[i][:, None] + D_ov[j][None, :])
        t2_ab = (B[i, sv] @ B[j, sv].T) / d2_ab
        t2[i, j] = t2_ab
        factor = 2 if i != j else 1
        e_corr += factor * (t2_ab * (2 * t2_ab - t2_ab.T) * d2_ab).sum()
        if i != j:
            t2[j, i] = t2_ab.T

print(f"MP2 e_corr {e_corr}")

intermediates = {
    "M1j": None,
    "M1oo": None,
    "M1aov": None,
    "M1bov": None,
    "M1vv": None,
    "M2a": None,
    "M2b": None,
}

for i in range(100):
    print(f"=== iteration {i} ===")
    get_riccsd_intermediates_1(intermediates, B, t1, t2)
    (e_corr, t1, t2), (delta_e_corr, delta_t1, delta_t2) = get_riccsd_amplitude(intermediates, B, mo_energy, e_corr, t1, t2)
    if delta_e_corr < 1e-7 and delta_t1 < 1e-5 and delta_t2 < 1e-5:
        break

MP2 e_corr -0.2027539177355094
=== iteration 0 ===
delta_e_corr 0.005239822061174626
delta_t1     0.04577406620457161
delta_t2     0.6288253059568694
e_corr       -0.20799373979668404
=== iteration 1 ===
delta_e_corr 0.0031049880275015596
delta_t1     0.00899702551326303
delta_t2     0.19035492947826982
e_corr       -0.2110987278241856
=== iteration 2 ===
delta_e_corr 0.0006920906765417456
delta_t1     0.00442466132512682
delta_t2     0.06188439259412255
e_corr       -0.21179081850072734
=== iteration 3 ===
delta_e_corr 0.0002674380057528236
delta_t1     0.0016531799635276863
delta_t2     0.02408455765575785
e_corr       -0.21205825650648016
=== iteration 4 ===
delta_e_corr 9.53670364023107e-05
delta_t1     0.0007457361270473629
delta_t2     0.009718102230343598
e_corr       -0.21215362354288247
=== iteration 5 ===
delta_e_corr 3.803422412995694e-05
delta_t1     0.00033367856551322724
delta_t2     0.004124376139367448
e_corr       -0.21219165776701243
=== iteration 6 ===
delta_e_corr 1

## Save temporaries

In [21]:
def save_riccsd_temporaries(intermediates, B, mo_energy):
    t1 = mf_cc.t1
    t2 = mf_cc.t2

    os.makedirs("h2o-cc-pvdz", exist_ok=True)
    np.save("h2o-cc-pvdz/mo_coeff.npy", np.ascontiguousarray(mf.mo_coeff))
    np.save("h2o-cc-pvdz/mo_occ.npy", mf.mo_occ)
    np.save("h2o-cc-pvdz/mo_energy.npy", mf.mo_energy)
    np.save("h2o-cc-pvdz/t1.npy", t1)
    np.save("h2o-cc-pvdz/t2.npy", t2)
    np.save("h2o-cc-pvdz/cderi.npy", mf.with_df._cderi)
    
    get_riccsd_intermediates_1(intermediates, B, t1, t2)
    get_riccsd_intermediates_2(intermediates, B, t1, t2)

    np.save("h2o-cc-pvdz/M1j.npy", intermediates["M1j"])
    np.save("h2o-cc-pvdz/M1oo.npy", intermediates["M1oo"])
    np.save("h2o-cc-pvdz/M1aov.npy", intermediates["M1aov"])
    np.save("h2o-cc-pvdz/M1bov.npy", intermediates["M1bov"])
    np.save("h2o-cc-pvdz/M1vv.npy", intermediates["M1vv"])
    np.save("h2o-cc-pvdz/M2a.npy", intermediates["M2a"])
    np.save("h2o-cc-pvdz/M2b.npy", intermediates["M2b"])

    RHS1 = np.zeros([nocc, nvir])
    get_riccsd_residue_1(RHS1, intermediates, B, t1, t2)
    np.save("h2o-cc-pvdz/RHS1.npy", RHS1)

    RHS2 = np.zeros([nocc, nocc, nvir, nvir])
    RHS2_tmp = np.zeros([nocc, nocc, nvir, nvir])
    get_riccsd_residue_2_lt2_contract(RHS2_tmp, intermediates, B, t1, t2)
    RHS2 += RHS2_tmp
    np.save("h2o-cc-pvdz/RHS2-lt2_contract.npy", RHS2_tmp)
    
    RHS2_tmp = np.zeros([nocc, nocc, nvir, nvir])
    get_riccsd_residue_2_direct_dot(RHS2_tmp, intermediates, B, t1, t2)
    RHS2 += RHS2_tmp
    np.save("h2o-cc-pvdz/RHS2-direct_dot.npy", RHS2_tmp)
    
    RHS2_tmp = np.zeros([nocc, nocc, nvir, nvir])
    get_riccsd_residue_2_o3v3(RHS2_tmp, intermediates, B, t1, t2)
    RHS2 += RHS2_tmp
    np.save("h2o-cc-pvdz/RHS2-o3v3.npy", RHS2_tmp)
    
    RHS2_tmp = np.zeros([nocc, nocc, nvir, nvir])
    get_riccsd_residue_2_o2v4(RHS2_tmp, intermediates, B, t1, t2)
    RHS2 += RHS2_tmp
    np.save("h2o-cc-pvdz/RHS2-o2v4.npy", RHS2_tmp)
    
    RHS2_tmp = np.zeros([nocc, nocc, nvir, nvir])
    get_riccsd_residue_2_o4v2(RHS2_tmp, intermediates, B, t1, t2)
    RHS2 += RHS2_tmp
    np.save("h2o-cc-pvdz/RHS2-o4v2.npy", RHS2_tmp)

In [22]:
save_riccsd_temporaries(intermediates, B, mo_energy)